In [2]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
import metis
import networkit as nk
from networkit import vizbridges
import pickle
# import powerlaw

/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_91281/3872114279.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# 파일 불러오기
with open('../../data/df.pkl', 'rb') as f:
    df = pickle.load(f)


In [4]:
# 파일 불러오기
with open('../../data/df_qna.pkl', 'rb') as f:
    df_qna = pickle.load(f)

In [5]:
df['q_creationdate'].min()

Timestamp('2023-08-03 01:04:49.693000')

In [6]:
df_gephi = df[['q_owneruserid', 'a_owneruserid']]

In [7]:
df_gephi.columns = ['Source', 'Target']

In [8]:
node = np.unique(np.append(df_gephi['Source'].values, df_gephi['Target'].values))

In [9]:
# 질문자->답변자 로 edgelist 생성
edgelist = list(zip(df_gephi['Source'], df_gephi['Target']))

In [10]:
di_github = nx.DiGraph()
di_github.add_nodes_from(node)
di_github.add_edges_from(edgelist)

github = nx.from_pandas_edgelist(df,  source='q_owneruserid', target='a_owneruserid')

In [11]:
di_github_nk = nk.nxadapter.nx2nk(di_github)
github_nk = nk.nxadapter.nx2nk(github)

In [12]:
print(di_github_nk.numberOfNodes(), di_github_nk.numberOfEdges())

11392 11996


### Core Decomposition

In [13]:
rm_github_nk = github_nk.removeSelfLoops()

In [14]:
type(rm_github_nk)

NoneType

In [15]:
coreDec = nk.centrality.CoreDecomposition(github_nk)
coreDec.run()
# direct 라 안되는데
# untimeError: Core Decomposition implementation does not support graphs with self-loops. Call Graph.removeSelfLoops() first.

In [16]:
# nk.gephi.pyclient.GephiClient(url='http://116.45.16.200:5999/workspace0', autoflush=False)
# gsc = nk.gephi.streaming.GephiStreamingClient(url='http://116.45.16.200:5999/workspace1')
# gsc = nk.gephi.streaming.GephiStreamingClient(url='http://143.248.106.43:7777/workspace1')
# gsc = nk.gephi.streaming.GephiStreamingClient(url='http://143.248.106.43:7777/workspace2')
gsc = nk.gephi.streaming.GephiStreamingClient(url='http://localhost:7777/workspace1')


In [17]:
# test = nk.gephi.streaming.GephiStreamingClient(url='http://localhost:8080/workspace1')

In [18]:
# test = nk.gephi.streaming.GephiStreamingClient(url='http://116.45.16.200:8080/workspace2')

In [19]:
# test.exportGraph(di_github_nk)

In [20]:
gsc.exportGraph(di_github_nk)

Could not connect to the gephi streaming plugin.
Did you start the streaming master server in gephi and provide the name of your workspace?
If the workspace is named 'Workspace 0', the corresponding url is http://localhost:8080/workspace0 (adapt port)


In [21]:
# nk.vizbridges.widgetFromGraph(github_nk, dimension = nk.vizbridges.Dimension.Two, nodeScores = coreDec.scores())

In [22]:
di_github_nk
# 1762

In [23]:
list(di_github.nodes())[1762]

2376954

In [24]:
list(di_github.nodes())[5115]

10542284

In [31]:
list(di_github.nodes())[7498]

17758716

In [29]:
# java spring 내에서 가장 답변을 많이 한 17758716 
# in degree 가 높은 사람들의 질문에 답하기 보다는 상대적으로 낮은 사람들에게 답을 해줬다 
# why?
df[(df['q_owneruserid']==list(di_github.nodes())[7498])|(df['a_owneruserid']==list(di_github.nodes())[7498])]

,q_id,q_posttypeid,q_acceptedanswerid,q_parentid,q_creationdate,q_score,q_viewcount,q_owneruserid,q_title,q_tags,...,a_parentid,a_creationdate,a_score,a_viewcount,a_owneruserid,a_title,a_tags,a_answercount,a_commentcount,a_reputation
25,77574127,1,NaN,None,2023-11-29 19:52:09.290,0,71,1919389,Java - Exception without interrupting program,java exception arraylist,...,77574127,2023-11-29 20:07:08.440,0,None,17758716,None,None,None,0,1
173,77565205,1,NaN,None,2023-11-28 15:40:50.723,2,73,23001366,How to get index of a string in an a list,java json codeigniter encoding,...,77565205,2023-11-28 18:39:31.647,0,None,17758716,None,None,None,0,1
190,77566706,1,NaN,None,2023-11-28 19:36:17.640,0,54,13363175,Collectors example question on Lambda functions,java,...,77566706,2023-11-28 20:37:27.923,0,None,17758716,None,None,None,0,1
203,77562561,1,NaN,None,2023-11-28 09:13:27.563,-1,51,22998701,"Scanning the next line, but only to a certain ...",java string java.util.scanner maxlength,...,77562561,2023-11-28 21:32:18.010,0,None,17758716,None,None,None,0,1
205,77567742,1,NaN,None,2023-11-28 23:26:37.360,0,51,20028948,How to advance row-major and wrap around rows ...,java arrays multidimensional-array char,...,77567742,2023-11-29 02:17:31.923,1,None,17758716,None,None,None,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11887,76833588,1,NaN,None,2023-08-04 06:54:08.890,0,56,1781423,How to access and process Fields of Type Map i...,java reflection,...,76833588,2023-08-07 04:56:39.560,0,None,17758716,None,None,None,1,1
11914,76843490,1,76843513.0,None,2023-08-05 20:49:43.767,0,83,9784846,Compare string with toString() representation ...,java enums switch-statement,...,76843490,2023-08-05 21:51:28.217,0,None,17758716,None,None,None,5,1
11924,76843148,1,77034207.0,None,2023-08-05 19:07:44.470,4,1400,12905411,Find longest sum for array and given days,java algorithm,...,76843148,2023-08-05 22:51:58.427,0,None,17758716,None,None,None,5,1
11930,76841455,1,76842619.0,None,2023-08-05 11:37:08.453,1,79,7768127,How to match a complex string using Negative l...,java regex,...,76841455,2023-08-05 23:28:30.267,0,None,17758716,None,None,None,0,1


In [28]:
df[(df['q_owneruserid']==list(di_github.nodes())[7221])|(df['a_owneruserid']==list(di_github.nodes())[7221])]

,q_id,q_posttypeid,q_acceptedanswerid,q_parentid,q_creationdate,q_score,q_viewcount,q_owneruserid,q_title,q_tags,...,a_parentid,a_creationdate,a_score,a_viewcount,a_owneruserid,a_title,a_tags,a_answercount,a_commentcount,a_reputation
5847,77228792,1,NaN,None,2023-10-04 10:30:49.673,0,43,16802243,getting ElementClickInterceptedException in th...,java selenium-webdriver selenium-chromedriver ...,...,77228792,2023-10-04 11:43:47.417,0,None,1751801,None,None,None,2,1497
6649,77177966,1,NaN,None,2023-09-26 07:47:04.080,-1,37,16802243,I can't able to create a node in my local machine,java selenium-webdriver selenium-chromedriver ...,...,77177966,2023-09-26 07:53:16.117,0,None,22531817,None,None,None,0,119


### 고군분투한 포인트  
시각화 해서 노드 간의 전반적인 관계를 보고 싶은데, 현재 환경에서 바로 네트워크 그래프를 볼 수 있는 방법이 없음...  
* 서브 그래프를 그리는 방법
    * centrality기반으로 주요노드들 추출한 다음에 해당 노드를 포함하는 그래프를 서브 그래프로 그려보기(closeness/betweenness)
    * community detection 결과로 나뉘어진 그룹기준으로 서브 그래프 그려보기(방향 없는 그래프로 변환 후 그려야 함)

